In [2]:
import json

# Load the buoy data
with open('buoy_data (1).json') as f:
    buoy_data = json.load(f)

# Initialize the GeoJSON structure
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for buoy_id, buoy_info in buoy_data.items():
    try:
        # Check for valid latitude and longitude values
        lat_str = buoy_info.get('lat', '')
        long_str = buoy_info.get('long', '')
        
        if not lat_str or not long_str:
            print(f"Skipping buoy {buoy_id} due to missing coordinates.")
            continue
        
        # Convert latitude and longitude to float
        latitude = float(lat_str[:-1]) * (1 if lat_str[-1] == 'N' else -1)
        longitude = float(long_str[:-1]) * (1 if long_str[-1] == 'E' else -1)

        # Create a feature for each buoy
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [longitude, latitude]
            },
            "properties": {
                "id": buoy_id,
                "data": buoy_info['years']
            }
        }
        geojson_data["features"].append(feature)

    except ValueError as e:
        print(f"Skipping buoy {buoy_id} due to conversion error: {e}")

# Save as GeoJSON
with open('buoy_data.geojson', 'w') as f:
    json.dump(geojson_data, f, indent=2)

Skipping buoy PGBP7 due to missing coordinates.
Skipping buoy 51209 due to missing coordinates.
Skipping buoy 52202 due to missing coordinates.
Skipping buoy 52211 due to missing coordinates.
Skipping buoy WAKP8 due to missing coordinates.
Skipping buoy KWJP8 due to missing coordinates.
Skipping buoy APRP7 due to missing coordinates.
Skipping buoy 52200 due to missing coordinates.


In [6]:

# Load the GeoJSON data from the user's uploaded file
file_path = 'buoy_data.geojson'
with open(file_path, 'r') as f:
    geojson_data = json.load(f)

# Sample check and restructuring of each feature's properties
for feature in geojson_data['features']:
    # Extract basic properties
    buoy_id = feature['properties'].get('id')
    lat = feature['properties'].get('lat')
    long = feature['properties'].get('long')
    years_data = feature['properties'].get('years')

    # If `years` data exists, split it into separate properties for easy access
    if years_data:
        # Flatten the structure by moving yearly data under each metric
        wvht = {year: data['wvht'] for year, data in years_data.items()}
        dpd = {year: data['dpd'] for year, data in years_data.items()}
        wtmp = {year: data['wtmp'] for year, data in years_data.items()}
        power = {year: data['Power'] for year, data in years_data.items()}
        
        # Replace the years_data with separate metric-based properties
        feature['properties'] = {
            "id": buoy_id,
            "lat": lat,
            "long": long,
            "wvht": wvht,
            "dpd": dpd,
            "wtmp": wtmp,
            "Power": power
        }

# Save the modified GeoJSON
output_path = 'buoy_data.geojson'
with open(output_path, 'w') as f:
    json.dump(geojson_data, f, indent=2)

output_path


'buoy_data.geojson'

In [13]:
import json

# Load original GeoJSON data
with open('buoy_data.geojson', 'r') as file:
    data = json.load(file)

# Check if 'features' key exists
if "features" in data:
    flattened_features = []

    for feature in data["features"]:
        # Access properties and data dictionary for each feature
        properties = feature.get("properties", {})
        buoy_id = properties.get("id", "Unknown")
        lat = feature["geometry"]["coordinates"][1]  # Assuming lat is second coordinate
        lon = feature["geometry"]["coordinates"][0]  # Assuming lon is first coordinate
        years_data = properties.get("data", {})

        # Flatten year data with default values for missing properties
        for year, year_data in years_data.items():
            flattened_feature = {
                "id": buoy_id,
                "latitude": lat,
                "longitude": lon,
                "year": year,
                "wvht": year_data.get("wvht", "N/A"),
                "dpd": year_data.get("dpd", "N/A"),
                "wtmp": year_data.get("wtmp", "N/A"),
                "Power": year_data.get("Power", "N/A")
            }
            flattened_features.append(flattened_feature)

    # Construct the new flattened GeoJSON structure
    flattened_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [feature["longitude"], feature["latitude"]]
                },
                "properties": {
                    "id": feature["id"],
                    "year": feature["year"],
                    "wvht": feature["wvht"],
                    "dpd": feature["dpd"],
                    "wtmp": feature["wtmp"],
                    "Power": feature["Power"]
                }
            }
            for feature in flattened_features
        ]
    }

    # Save the flattened data back to a new file
    with open('flattened_buoy_data.geojson', 'w') as file:
        json.dump(flattened_data, file)

else:
    print("Unexpected data structure. Please verify the content of 'buoy_data.geojson'.")


In [15]:
#filtered without 99 values (i.e. no height and period)
# Load the original GeoJSON data
with open('buoy_data.geojson', 'r') as file:
    data = json.load(file)

# Check if 'features' key exists
if "features" in data:
    flattened_features = []

    for feature in data["features"]:
        # Access properties and data dictionary for each feature
        properties = feature.get("properties", {})
        buoy_id = properties.get("id", "Unknown")
        lat = feature["geometry"]["coordinates"][1]  # Assuming lat is second coordinate
        lon = feature["geometry"]["coordinates"][0]  # Assuming lon is first coordinate
        years_data = properties.get("data", {})

        # Check if all years have wvht and dpd set to 99
        all_99 = all(year_data.get("wvht") == 99 and year_data.get("dpd") == 99 for year_data in years_data.values())
        
        # Skip this buoy if all years have both wvht and dpd as 99
        if all_99:
            continue

        # Flatten year data with default values for missing properties
        for year, year_data in years_data.items():
            flattened_feature = {
                "id": buoy_id,
                "latitude": lat,
                "longitude": lon,
                "year": year,
                "wvht": year_data.get("wvht", "N/A"),
                "dpd": year_data.get("dpd", "N/A"),
                "wtmp": year_data.get("wtmp", "N/A"),
                "Power": year_data.get("Power", "N/A")
            }
            flattened_features.append(flattened_feature)

    # Construct the new flattened GeoJSON structure
    flattened_data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [feature["longitude"], feature["latitude"]]
                },
                "properties": {
                    "id": feature["id"],
                    "year": feature["year"],
                    "wvht": feature["wvht"],
                    "dpd": feature["dpd"],
                    "wtmp": feature["wtmp"],
                    "Power": feature["Power"]
                }
            }
            for feature in flattened_features
        ]
    }

    # Save the filtered and flattened data back to a new file
    with open('flattened_buoy_data.geojson', 'w') as file:
        json.dump(flattened_data, file)

else:
    print("Unexpected data structure. Please verify the content of 'buoy_data.geojson'.")
